In [1]:
from urllib.parse import quote_plus
from string import ascii_uppercase
import os
import re
import pandas as pd
import sqlite3

In [2]:
file_name = "總統-A05-4-候選人得票數一覽表-各投開票所(臺北市).xls"
file_url = quote_plus(file_name)
spreadsheet_url = "https://taiwan-election-data.s3-ap-northeast-1.amazonaws.com/presidential_2020/{}".format(file_url)
xl = pd.ExcelFile(spreadsheet_url)
print(xl.sheet_names)

['臺北市']


In [3]:
df = pd.read_excel(spreadsheet_url)
df.head()

,第15任總統副總統選舉候選人在臺北市各投開票所得票數一覽表,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,鄉(鎮、市、區)別,村里別,投票所別,各組候選人得票情形,NaN,NaN,有效票數A\nA=1+2+...+N,無效票數B,投票數C\nC=A+B,已領未投票數\nD\nD=E-C,發出票數E\nE=C+D,用餘票數F,選舉人數G\nG=E+F,投票率H\nH=C÷G
1,NaN,NaN,NaN,(1)\n宋楚瑜\n余湘,(2)\n韓國瑜\n張善政,(3)\n蔡英文\n賴清德,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,總 計,NaN,NaN,"70,769","685,830","875,854","1,632,453","21,381","1,653,834",143,"1,653,977","513,287","2,167,264",76.3098


## **We can specify which sheet to import via sheet_name**

In [ ]:
file_name = "區域立委-A05-2-得票數一覽表(臺北市).xls"
file_url = quote_plus(file_name)
spreadsheet_url = "https://taiwan-election-data.s3-ap-northeast-1.amazonaws.com/legislative_2020/{}".format(file_url)
xl = pd.ExcelFile(spreadsheet_url)
print(xl.sheet_names)

regional_legislator_taipei_city = {}
for sheet_name in xl.sheet_names:
    regional_legislator_taipei_city[sheet_name] = pd.read_excel(spreadsheet_url, sheet_name=sheet_name)
for k, v in regional_legislator_taipei_city.items():
    print("Shape of {} : {}".format(k, v.shape))

# **Using skiprows to skip those combined cells while importing spreadsheets**

In [ ]:
file_name = "總統-A05-4-候選人得票數一覽表-各投開票所(臺北市).xls"
file_url = quote_plus(file_name)
spreadsheet_url = "https://taiwan-election-data.s3-ap-northeast-1.amazonaws.com/presidential_2020/{}".format(file_url)
df = pd.read_excel(spreadsheet_url, skiprows=[0, 1, 3, 4], thousands=',')
df.head()

# **Updating columns attributes with new columns**

In [ ]:
n_cols = df.columns.size
n_candidates = n_cols - 11
id_vars = ['town', 'village', 'office']
candidates = list(df.columns[3:(3 + n_candidates)])
office_cols = list(ascii_uppercase[:8])
col_names = id_vars + candidates + office_cols
df.columns = col_names

print(n_candidates)
print(candidates)
print(office_cols)
print(col_names)

# **ffill: propagate last valid observation forward to next valid.**

In [ ]:
filled_towns = df['town'].fillna(method='ffill')
df = df.assign(town=filled_towns)
df.head()

df = df.dropna()

# *Removing extra spaces in district via strip*

In [ ]:
print(df['town'].unique())
stripped_strict = df['town'].str.replace("\u3000", "")
df = df.assign(district=stripped_strict)
print(df['town'].unique())

# **Pivoting candidate columns into a new pair of variables via melt**
`melt`

In [ ]:
df = df.drop(labels=office_cols, axis=1)
df_long = pd.melt(df,
                  id_vars=id_vars,
                  var_name='candidate_info',
                  value_name='votes'
                 )
df_long.head()

# *總整*

In [ ]:
def tidy_dataframe(df):
    # updating columns attributes 
    n_cols = df.columns.size
    n_candidates = n_cols - 11
    id_vars = ['town', 'village', 'office']
    candidates = list(df.columns[3:(3 + n_candidates)])
    office_cols = list(ascii_uppercase[:8])
    col_names = id_vars + candidates + office_cols
    df.columns = col_names
    # forward-fill district values
    filled_towns = df['town'].fillna(method='ffill')
    df = df.assign(town=filled_towns)
    # removing summations
    df = df.dropna()
    # removing extra spaces
    stripped_towns = df['town'].str.replace("\u3000", "")
    df = df.assign(town=stripped_towns)
    # pivoting
    df = df.drop(labels=office_cols, axis=1)
    tidy_df = pd.melt(df,
                      id_vars=id_vars,
                      var_name='candidate_info',
                      value_name='votes'
                     )
    return tidy_df

def adjust_presidential(df):
    # split candidate information into 2 columns
    candidate_info_df = df['candidate_info'].str.split("\n", expand=True)
    numbers = candidate_info_df[0].str.replace("\(|\)", "")
    candidates = candidate_info_df[1].str.cat(candidate_info_df[2], sep="/")
    # re-arrange columns
    df = df.drop(labels='candidate_info', axis=1)
    df['number'] = numbers
    df['candidate'] = candidates
    df['office'] = df['office'].astype(int)
    df = df[['county', 'town', 'village', 'office', 'number', 'candidate', 'votes']]
    return df

files = [i for i in os.listdir("presidential-2020/") if not i.startswith('.')] # to skip those hidden files
counties = [re.split("\(|\)", f)[1] for f in files]
print(files)
print(counties)

counties = ['宜蘭縣', '彰化縣', '金門縣', '桃園市', '苗栗縣', '臺南市', '雲林縣', '南投縣', '高雄市', '臺北市', '新北市', '花蓮縣', '新竹市', '新竹縣', '基隆市', '連江縣', '嘉義縣', '嘉義市', '屏東縣', '澎湖縣', '臺東縣', '臺中市']
presidential = pd.DataFrame()
for county in counties:
    file_name = "總統-A05-4-候選人得票數一覽表-各投開票所({}).xls".format(county)
    file_url = quote_plus(file_name)
    spreadsheet_url = "https://taiwan-election-data.s3-ap-northeast-1.amazonaws.com/presidential_2020/{}".format(file_url)
    # skip those combined cells
    df = pd.read_excel(spreadsheet_url, skiprows=[0, 1, 3, 4], thousands=',')
    tidy_df = tidy_dataframe(df)
    # appending dataframe of each city/county
    tidy_df['county'] = county
    presidential = presidential.append(tidy_df)
    print("Tidying {}".format(file_name))
presidential = presidential.reset_index(drop=True) # reset index for the appended dataframe

In [ ]:
legislative_regional = pd.concat([regional, indigenous], axis=0)
legislative_regional = legislative_regional.reset_index(drop=True)

# **Extra resources**

In [ ]:
from taiwan_election_2020 import TaiwanElection2020

tw_election = TaiwanElection2020()
presidential = tw_election.generate_presidential()
legislative_regional = tw_election.generate_legislative_regional()
legislative_at_large = tw_election.generate_legislative_at_large()

In [ ]:
presidential.to_csv('presidential.csv', index=False)
legislative_regional.to_csv('legislative_regional.csv', index=False)
legislative_at_large.to_csv('legislative_at_large.csv', index=False)

# or
conn = sqlite3.connect('tw-election-2020.db')
presidential.to_sql('presidential', con=conn, index=False, if_exists='replace')
legislative_regional.to_sql('legislative_regional', con=conn, index=False, if_exists='replace')
legislative_at_large.to_sql('legislative_at_large', con=conn, index=False, if_exists='replace')